In [1]:
import pandas as pd
df=pd.read_csv('Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [2]:
cols = ['RowNumber','CustomerId','Surname']
df.drop(cols,inplace=True,axis=1)


In [3]:
from sklearn.preprocessing import OrdinalEncoder

object_cols = ["Geography", "Gender"]

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])

In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [5]:
# Assuming you have already defined X and y
# X = ... (your input data)
# y = ... (your corresponding target labels)

# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression classifier
clf = LogisticRegression(max_iter=1000)

# Define the number of folds for cross-validation
num_folds = 10

# Create a k-fold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation and get accuracy scores for each fold
accuracy_scores = cross_val_score(clf, X, y, cv=kfold)

# Print accuracy scores for each fold
print("Accuracy scores for each fold:")
for i, accuracy in enumerate(accuracy_scores, start=1):
    print(f"Fold {i}: {accuracy:.2f}")

# Calculate and print the mean accuracy across all folds
mean_accuracy = np.mean(accuracy_scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")

# Fit the model on the entire dataset
clf.fit(X, y)

# Generate predictions for the input data X
y_pred = clf.predict(X)

# Get the probabilities for each class
y_pred_probabilities = clf.predict_proba(X)

# Get the probability of the first class (index 0) for each sample
y_pred_prob = y_pred_probabilities[:, 1]

# Create a new DataFrame with 'y_pred', 'y', and 'y_pred_prob' as columns
result_df = pd.DataFrame({
    'y_pred': y_pred,
    'y': y,
    'y_pred_prob': y_pred_prob
})

# Now you have a new DataFrame 'result_df' with the predicted labels 'y_pred',
# the corresponding true labels 'y', and the probability of the first class 'y_pred_prob' for each sample


Accuracy scores for each fold:
Fold 1: 0.81
Fold 2: 0.79
Fold 3: 0.80
Fold 4: 0.77
Fold 5: 0.77
Fold 6: 0.79
Fold 7: 0.81
Fold 8: 0.77
Fold 9: 0.78
Fold 10: 0.80
Mean Accuracy: 0.79


In [6]:
incorrect_predictions_df = result_df[result_df['y_pred'] != result_df['y']]
incorrect_prediction_indexes = incorrect_predictions_df.index
incorrect_prediction_features = X.iloc[incorrect_prediction_indexes]

# Add the feature columns to the incorrect_predictions_df
incorrect_predictions_df = pd.concat([incorrect_predictions_df, incorrect_prediction_features], axis=1)

In [7]:
def average_ranking_loss_with_penalty(y_true, rankings):
    sorted_indices = np.argsort(rankings)
    sorted_true = y_true[sorted_indices]
    sorted_ranking = rankings[sorted_indices]
    
    true_indices = np.where(sorted_true == 1)[0]
    false_indices = np.where(sorted_true == 0)[0]
    
    false_rankings = sorted_ranking[false_indices]
    higher_rankings = false_rankings > np.expand_dims(sorted_ranking[true_indices], axis=1)
    equal_rankings = false_rankings == np.expand_dims(sorted_ranking[true_indices], axis=1)
    
    PENNi_sum = np.sum(higher_rankings, axis=1) + 0.5 * np.sum(equal_rankings, axis=1)
    numerator_sum = np.sum(sorted_true[true_indices] * PENNi_sum)
    denominator_sum = np.sum(y_true)
    
    return numerator_sum / denominator_sum

In [8]:
y_true = np.array(result_df.y)
rankings = np.array(result_df.y_pred_prob)
average_ranking_loss_with_penalty(y_true,rankings)

2623.911143838979

In [9]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y, y_pred_prob)
print(f"AUC Score: {auc_score:.2f}")

AUC Score: 0.67


In [15]:
df1=df.loc[df['Age']>=55]
X1 = df1.iloc[:,:-1]
y1 = df1.iloc[:,-1]

In [16]:
y_pred1 = clf.predict(X1)

# Get the probabilities for each class
y_pred_probabilities1 = clf.predict_proba(X1)

# Get the probability of the first class (index 0) for each sample
y_pred_prob1 = y_pred_probabilities1[:, 1]

# Create a new DataFrame with 'y_pred', 'y', and 'y_pred_prob' as columns
result_df1 = pd.DataFrame({
    'y_pred': y_pred1,
    'y': y1,
    'y_pred_prob': y_pred_prob1
})


In [17]:
y_true1 = np.array(result_df1.y)
rankings1 = np.array(result_df1.y_pred_prob)
average_ranking_loss_with_penalty(y_true1,rankings1)

302.60233918128654

In [18]:
auc_score1 = roc_auc_score(y1, y_pred_prob1)
auc_score1

0.4396252978124323